# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
#!pip install nltk
#!pip install plotly

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aevdochuk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aevdochuk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\aevdochuk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# import libraries
from sqlalchemy import create_engine
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.svm import LinearSVC

import pickle
import warnings
warnings.filterwarnings('ignore') 

In [4]:
# load data from database
db_path = 'disaster_response.db'
engine = create_engine('sqlite:///' + db_path)
table_name = db_path.replace(".db","") + "_table"
df = pd.read_sql_table(table_name, engine)

In [5]:
df.head()   

,id,message,original,genre,_merge,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,both,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,both,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,both,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,both,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,both,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Create X and y 
X = df['message']
y = df.iloc[:,5:]

In [7]:
categories = df.iloc[:,5:].columns

In [8]:
categories

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [9]:
category_boolean = (df.iloc[:,5:] != 0).sum().values

In [10]:
category_boolean

array([20094,  4474,   118, 10860,  2084,  1313,   724,   471,   860,
        1672,  2923,  2314,   405,   604,   298,   875,  1194,  3446,
        1705,  1201,  1333,   532,   159,   283,   120,   309,  1151,
        7297,  2155,  2443,   282,  2455,   530,  1376,  5075],
      dtype=int64)

In [11]:
df.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):

    regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # apply regex to get urls
    found_urls = re.findall(regex, text)
    
    # replace urls with placeholder
    for url in found_urls:
        text = text.replace(url,'placeholder')
    
    lemmatizer = WordNetLemmatizer()

    # confert text into tokens
    tokens = word_tokenize(text)
    
    # go through each token and do the following: lemmatize, normalize case, and remove leading/trailing white space
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
# pipeline_a uses Random Forest Classifier


pipeline_a = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf1', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

In [14]:
# pipeline_b uses Linear Support Vector Machines Classifier
pipeline_b = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf2',  MultiOutputClassifier(estimator=LinearSVC(max_iter=100000, random_state=42))) 
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
df.head()

,id,message,original,genre,_merge,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,both,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,both,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,both,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,both,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,both,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline_a.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000002932852E3A0>)),
                ('clf1',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred_a = pipeline_a.predict(X_test)

# accuracy of the training
print("The accuracy of the training is: ")
print(accuracy_score(y_train, pipeline_a.predict(X_train))*100)
print(" ")

# accuracy of the test
print("The accuracy of the test is: ")
print(accuracy_score(y_test, y_pred_a)*100)
print(" ")

# classification report
print("Results of the classification report:")
print(classification_report(y_test, y_pred_a, target_names=y.columns.values))

The accuracy of the training is: 
99.71010070186146
 
The accuracy of the test is: 
24.36679890143424
 
Results of the classification report:
                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4980
               request       0.90      0.44      0.59      1115
                 offer       0.00      0.00      0.00        23
           aid_related       0.80      0.60      0.69      2744
          medical_help       0.58      0.04      0.07       551
      medical_products       0.84      0.06      0.12       332
     search_and_rescue       0.75      0.02      0.03       188
              security       0.50      0.01      0.02       121
              military       0.71      0.02      0.04       242
                 water       0.88      0.20      0.33       409
                  food       0.88      0.42      0.57       695
               shelter       0.90      0.21      0.34       588
              clothing   

In [18]:
pipeline_a.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'clf1', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__norm', 'vect__preprocessor', 'vect__smooth_idf', 'vect__stop_words', 'vect__strip_accents', 'vect__sublinear_tf', 'vect__token_pattern', 'vect__tokenizer', 'vect__use_idf', 'vect__vocabulary', 'clf1__estimator__bootstrap', 'clf1__estimator__ccp_alpha', 'clf1__estimator__class_weight', 'clf1__estimator__criterion', 'clf1__estimator__max_depth', 'clf1__estimator__max_features', 'clf1__estimator__max_leaf_nodes', 'clf1__estimator__max_samples', 'clf1__estimator__min_impurity_decrease', 'clf1__estimator__min_samples_leaf', 'clf1__estimator__min_samples_split', 'clf1__estimator__min_weight_fraction_leaf', 'clf1__estimator__n_estimators', 'clf1__estimator__n_jobs', 'clf1__estimator__oob_score', 'clf1__estimator__random_state', 'clf1__

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters = {'clf1__estimator__max_depth': [10, 50, None],
              'clf1__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline_a, param_grid=parameters, 
                            n_jobs=-1,
                            cv=5, refit=True, return_train_score=True, verbose=10)

cv.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x000002932852E3A0>)),
                                       ('clf1',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))]),
             n_jobs=-1,
             param_grid={'clf1__estimator__max_depth': [10, 50, None],
                         'clf1__estimator__min_samples_leaf': [2, 5, 10]},
             return_train_score=True, verbose=10)

In [20]:
print("The best hyperparameters based on Grid Search are:")
print(cv.best_params_)
print("")
print("Using these hyperparameters the average accuracy of the model is the following:")
print(cv.best_score_)

The best hyperparameters based on Grid Search are:
{'clf1__estimator__max_depth': None, 'clf1__estimator__min_samples_leaf': 2}

Using these hyperparameters the average accuracy of the model is the following:
0.21457673922225767


In [21]:
best_improved_clf = cv.best_estimator_

In [22]:
best_improved_clf.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000002932852E3A0>)),
                ('clf1',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_leaf=2,
                                                                        random_state=42)))])

In [23]:
y_pred_a_improved = best_improved_clf.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
# accuracy of the training
print("The accuracy of the training is: ")
print(accuracy_score(y_train, best_improved_clf.predict(X_train))*100)
print(" ")

# accuracy of the test
print("The accuracy of the test is: ")
print(accuracy_score(y_test, y_pred_a_improved)*100)
print(" ")

# classification report
print("classification report")
print(classification_report(y_test, y_pred_a_improved, target_names=y.columns.values))

The accuracy of the training is: 
32.11779066219103
 
The accuracy of the test is: 
21.574610924626185
 
classification report
                        precision    recall  f1-score   support

               related       0.79      0.98      0.88      4980
               request       0.93      0.29      0.44      1115
                 offer       0.00      0.00      0.00        23
           aid_related       0.80      0.62      0.70      2744
          medical_help       0.53      0.02      0.03       551
      medical_products       1.00      0.01      0.01       332
     search_and_rescue       0.00      0.00      0.00       188
              security       0.00      0.00      0.00       121
              military       0.50      0.00      0.01       242
                 water       1.00      0.02      0.04       409
                  food       0.91      0.15      0.26       695
               shelter       0.86      0.06      0.12       588
              clothing       1.00      0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
# use pipeline_a 
pipeline_b.fit(X_train, y_train)

y_pred_b = pipeline_b.predict(X_test)

# accuracy of the training
print("The accuracy of the training is: ")
print(accuracy_score(y_train, pipeline_b.predict(X_train))*100)
print(" ")

# accuracy of the test
print("The accuracy of the test is: ")
print(accuracy_score(y_test, y_pred_b)*100)
print(" ")

# classification report
print("Classification report")
print(classification_report(y_test, y_pred_b, target_names=y.columns.values))

The accuracy of the training is: 
72.00183094293561
 
The accuracy of the test is: 
28.51693622215441
 
Classification report
                        precision    recall  f1-score   support

               related       0.86      0.91      0.89      4980
               request       0.77      0.63      0.69      1115
                 offer       0.00      0.00      0.00        23
           aid_related       0.73      0.69      0.71      2744
          medical_help       0.61      0.28      0.39       551
      medical_products       0.65      0.28      0.39       332
     search_and_rescue       0.70      0.19      0.29       188
              security       0.00      0.00      0.00       121
              military       0.74      0.29      0.42       242
                 water       0.76      0.61      0.68       409
                  food       0.81      0.68      0.74       695
               shelter       0.79      0.55      0.64       588
              clothing       0.76      0.

### 9. Export your model as a pickle file

In [26]:
#The pipeline_a is exported
saved_model = pickle.dumps(pipeline_a)

In [27]:
clf_from_pickle = pickle.loads(saved_model)

In [28]:
clf_from_pickle

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000002932852E3A0>)),
                ('clf1',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

k neighbours classifier and random forest peroforms almost similarlry. We'll choose random forest for classification as the best model.

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.